## Day 22

https://adventofcode.com/2020/day/22

In [1]:
import collections
import itertools

In [2]:
import aocd

In [3]:
test_data = """
Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10
"""

In [4]:
def parse_data(data):
    lines = [line.split(':')[1] for line in data.split('\n\n')]
    deck1, deck2 = [
        [int(value) for value in s.split('\n') if value != '']
        for s in lines
    ]
    return deck1, deck2

In [5]:
# deck1, deck2 = parse_data(test_data)
deck1, deck2 = parse_data(aocd.get_data(day=22, year=2020))
len(deck1), len(deck2)

(25, 25)

### Solution to Part 1

In [6]:
def play(deck1, deck2):
    deck1 = collections.deque(deck1)
    deck2 = collections.deque(deck2)
    while deck1 and deck2:
        card1 = deck1.popleft()
        card2 = deck2.popleft()
        if card1 > card2:
            deck1.extend([card1, card2])
        elif card2 > card1:
            deck2.extend([card2, card1])
    return deck1, deck2

In [7]:
final1, final2 = play(deck1, deck2)
len(final1), len(final2)

(50, 0)

In [8]:
def score(deck1, deck2):
    winner = deck1 if deck1 else deck2
    total = 0
    for i, card in enumerate(reversed(winner)):
        total += card * (i + 1)
    return total

In [9]:
score(final1, final2)

35005

### Solution to Part 2

In [10]:
def _play_subgame(deck1, deck2, *, card1, card2, game, game_counter):
    subdeck1 = [
        card for i, card in enumerate(deck1)
        if i < card1
    ]
    subdeck2 = [
        card for i, card in enumerate(deck2)
        if i < card2
    ]
    return play_recursive(
        subdeck1,
        subdeck2,
        game=game,
        game_counter=game_counter,
    )

In [11]:
def play_recursive(
        deck1,
        deck2,
        *,
        verbose=False,
        game=None,
        game_counter=None,
):
    if game_counter is None:
        game_counter = itertools.count(1)
        game = next(game_counter)
    game_round = 0
    already_seen = set()

    deck1 = collections.deque(deck1)
    deck2 = collections.deque(deck2)
    while deck1 and deck2:
        game_round += 1
        if verbose:
            print(f'Game {game} Round {game_round}')
            print(f'Player 1 deck: {list(deck1)}')
            print(f'Player 2 deck: {list(deck2)}')
        
        configuration = (tuple(deck1), tuple(deck2))
        if configuration in already_seen:
            break
        already_seen.add(configuration)
        
        card1 = deck1.popleft()
        card2 = deck2.popleft()
        if verbose:
            print(f'Player 1 plays {card1}')
            print(f'Player 2 plays {card2}')
        
        if len(deck1) >= card1 and len(deck2) >= card2:
            if verbose:
                print('Playing a subgame to determine a winner...\n')
            player1_wins_subgame, _ = _play_subgame(
                deck1,
                deck2,
                card1=card1,
                card2=card2,
                  game=next(game_counter),
                game_counter=game_counter,
            )
            if verbose:
                print()
                print(f'...Back to game {game}')
            if player1_wins_subgame:
                winner = 'Player 1'
                deck1.extend([card1, card2])
            else:
                winner = 'Player 2'
                deck2.extend([card2, card1])
            if verbose:
                print(f'{winner} wins round {game_round} of game {game}')
        elif card1 > card2:
            winner = 'Player 1'
            deck1.extend([card1, card2])
        elif card2 > card1:
            winner = 'Player 2'
            deck2.extend([card2, card1])
        if verbose:
            print(f'{winner} wins round {game_round} of game {game}')

    winner = 'Player 1' if deck1 else 'Player 2'
    if verbose:
        print(f'Winner of game {game} is {winner}')
    return deck1, deck2

In [12]:
final1, final2 = play_recursive(deck1, deck2)
len(final1), len(final2)

(50, 0)

In [13]:
score(final1, final2)

32751